In [2]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf

from yolov5.detect import run
from util import resize_face, is_face_centered
from get_data import create_directories, build_search_url, download_extended_page, download_image, get_all_items

c:\Users\cdetr\Documents\GitHub\you-look-like\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
f = open('people.txt')
keywords = f.read().splitlines()
# keywords = ['omar sy', 'elon musk']
print("Number of keywords:", len(keywords))

In [ ]:
sufix = "face"

keywords = [keyword + " " + sufix for keyword in keywords]
search_keywords = [str(item) for item in keywords]

limit = 1400

main_directory = "data/cnn/download"

In [ ]:
i_search_keywoard = 0

while i_search_keywoard < len(search_keywords):
	search_term = search_keywords[i_search_keywoard]

	subdirectory_name = search_term
	create_directories(main_directory, subdirectory_name)

	url = build_search_url(search_term)
	raw_html = download_extended_page(url)

	get_all_items(raw_html, main_directory, subdirectory_name, limit)

	i_search_keywoard += 1

In [ ]:
directory = 'data/cnn/download'
save_directory = 'data/cnn/download_cropped'
for celebrity in os.listdir(directory):
    if not os.path.exists(f'{save_directory}/{celebrity}'):
        os.mkdir(f'{save_directory}/{celebrity}')
    for img in os.listdir(f'{directory}/{celebrity}'):
        out = run('weights/yolov5/best.pt', source=f'{directory}/{celebrity}/{img}', nosave=True, conf_thres=0.8)
        if out['num_pred'] == 1:
            if is_face_centered(out['frame'], out['box']):
                crop_im = resize_face(out['frame'], out['box'])
                cv2.imwrite(f'{save_directory}/{celebrity}/{img}', crop_im)

In [10]:
data_augmentor = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
]) 

directory = 'data/cnn/download_cropped'
for celebrity in os.listdir(directory):
    celebrity_images = os.listdir(f'{directory}/{celebrity}')
    num_of_img = len(celebrity_images)
    for i in range(500-num_of_img):
        idx = random.randint(0, num_of_img-1)
        img_path = f'{directory}/{celebrity}/{celebrity_images[idx]}'
        img = tf.keras.preprocessing.image.load_img(img_path)
        img = tf.cast(tf.expand_dims(img, 0), tf.float32)
        aug_img = data_augmentor(img)
        aug_img = np.reshape(aug_img.numpy(), (aug_img.shape[1], aug_img.shape[2], aug_img.shape[3]))
        cv2.imwrite(f'{img_path[:-4]}_augmented_{i}.jpg', cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB))